In [654]:
import json
import statistics

In [655]:
small = "swg"
big = "starwars"

In [656]:
mappings_file_small = "./_input/mappings/" + small + ".json"
mappings_file_big = "./_input/mappings/" + big + ".json"

gold_pairs_file = "./_input/gold_downloaded/" + small + "-" + big + ".json"
exact_match_file = "./_input/exact_match_deduplicated/" + small + "-" + big + ".json"
found_pairs_file = "./_input/found_pairs_deduplicated/" + small + "-" + big + ".txt"

In [657]:
with open(mappings_file_small) as file:
    mappings_small = {str(v): k for k, v in json.load(file).items()}
    mappings_small_reversed = {v: k for k, v in mappings_small.items()}

with open(mappings_file_big) as file:
    mappings_big = {str(v): k for k, v in json.load(file).items()}
    mappings_big_reversed = {v: k for k, v in mappings_big.items()}

with open(gold_pairs_file) as gpf:
    gold_pairs = json.load(gpf)

with open(found_pairs_file) as fpf:
    found_pairs = []
    found_pairs_score = []
    for line in fpf:
        num_1 = line.strip().split("###")[0]
        num_2 = line.strip().split("###")[1]
        num_3 = float(line.strip().split("###")[2])
        numbers_list = [num_1, num_2]
        found_pairs.append(numbers_list)
        numbers_list_score = [num_1, num_2, num_3]
        found_pairs_score.append(numbers_list_score)

gold_pairs_left = {pair[0] for pair in gold_pairs}
gold_pairs_right = {pair[1] for pair in gold_pairs}

with open(exact_match_file) as file:
    exact_match = json.load(file)

exact_match_left = {pair[0] for pair in exact_match}

In [658]:
def compute_precision_recall_f1(gold_input, found_input):

    gold_set = set(tuple(pair) for pair in gold_input)
    found_set = set(tuple(pair) for pair in found_input)

    true_positives = gold_set & found_set
    tp = len(true_positives)
    fn = len(gold_set - found_set)
    fp = len(found_set - gold_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1

In [659]:
def print_found_but_not_gold(gold_input, found_input):

    found_but_not_gold = [[str(pair[0]), str(pair[1])]
                          for pair in found_input if pair not in gold_input]

    for item in found_but_not_gold:
        print(item[0])
        print(item[1])
        print()

    print(len(found_but_not_gold))

In [660]:
def print_gold_not_found(gold_input, found_input):

    gold_not_found = [[str(pair[0]), str(pair[1])]
                          for pair in gold_input if pair not in found_input]

    for item in gold_not_found:
        print(item[0])
        print(item[1])
        print()

    print(len(gold_not_found))

In [661]:
def clean_found_pairs(found_input_score, threshold):

    temp_dict = dict()

    found_input_score = [item for item in found_input_score if item[0].split("/")[4] == item[1].split("/")[4]]

    for pair in found_input_score:
        if str(pair[1]) not in temp_dict:
            temp_dict[str(pair[1])] = pair[2]
        elif pair[2] > temp_dict[str(pair[1])]:
            temp_dict[str(pair[1])] = pair[2]

    output = []

    for pair in found_input_score:
        if ((pair[0] in gold_pairs_left or pair[1] in gold_pairs_right)
                and pair[2] > threshold
                and pair[2] == temp_dict[str(pair[1])]):
            output.append([pair[0], pair[1]])

    return output

In [662]:
def add_exact_match(found_input):

    output = []

    for em in exact_match:
        if em[0] in gold_pairs_left or em[1] in gold_pairs_right:
            output.append(em)

    for pair in found_input:
        if pair[0] not in exact_match_left:
            output.append(pair)

    return output

In [663]:
def calculate_threshold(found_input_score):

    temp_dict = dict()

    for pair in found_input_score:
        if str(pair[1]) not in temp_dict:
            temp_dict[str(pair[1])] = pair[2]
        elif pair[2] > temp_dict[str(pair[1])]:
            temp_dict[str(pair[1])] = pair[2]

    scores_list = []
    for em in exact_match:
        if str(em[1]) in temp_dict:
            scores_list.append(temp_dict[str(em[1])])

    # threshold_val = statistics.mean(scores_list)
    threshold_val = statistics.median(scores_list)

    return threshold_val

In [664]:
len(found_pairs)

4539

In [665]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.11897
Recall: 0.48171
F1 Score: 0.19081


In [666]:
# reranker_threshold = calculate_threshold(found_pairs_score)
# reranker_threshold = 1
reranker_threshold = -1
# reranker_threshold = -10
print(f"Threshold: {reranker_threshold:.5f}")

Threshold: -1.00000


In [667]:
found_pairs_cleaned = clean_found_pairs(found_pairs_score, reranker_threshold)

In [668]:
len(found_pairs_cleaned)

809

In [669]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs_cleaned)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.63659
Recall: 0.45941
F1 Score: 0.53368


In [670]:
found_pairs_cleaned_em = add_exact_match(found_pairs_cleaned)

In [671]:
len(found_pairs_cleaned_em)

1051

In [672]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs_cleaned_em)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.79163
Recall: 0.74219
F1 Score: 0.76611


In [673]:
print_found_but_not_gold(gold_pairs, found_pairs_cleaned_em)

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pet
http://dbkwik.webdatacommons.org/starwars.wikia.com/class/pet

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Durindfire
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Durindfire

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Empire_Day
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Empire_Day

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Architect
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Architect

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Necklace
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Necklace

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Belbullab-22
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/B22

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Life_Day
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Life_Day

http://dbkwik.webdatacommons.org/swg.wikia.com/resou

In [674]:
print_gold_not_found(gold_pairs, found_pairs_cleaned_em)

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Target_Dummy
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/AB-US3

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Black_Sun_AEG-77_Vigo_Gunship
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/AEG-77_Vigo

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Agrilat_Swamp_Track
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Agrilat_Swamp_Circuit

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lt._Akal_Colzet
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Akal_Colzet

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Speak_with_Major_Walden
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Alana_Walden

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Field_Commander_Alozen
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Alozen

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Aludium_pu36_(Schematic)
http://dbkwi